In [1]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
import os
import datetime as dt

#sns.set(rc={'figure.figsize':(19, 13), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220220'
}

In [2]:
bot = telegram.Bot(token = '5291500178:AAFQqAVPSG4ad90QdyE-IaZH-Ai8ow2oxFs')
#updates = bot.getUpdates()
#print(updates[-1])

In [3]:
#chat_id = 1295860693
chat_id = -1001735251250

In [4]:
q = 'select toDate(time) as date, user_id, action  from simulator_20220220.feed_actions where toDate(time) > today()-8 and toDate(time) < today()'

df_feed = pandahouse.read_clickhouse(q, connection=connection)

In [6]:
df_feed

,date,user_id,action
0,2022-03-22,63830,view
1,2022-03-22,126860,view
2,2022-03-22,128603,view
3,2022-03-22,130058,view
4,2022-03-22,147928,view
...,...,...,...
5075597,2022-03-26,17243,like
5075598,2022-03-26,32723,view
5075599,2022-03-26,53128,view
5075600,2022-03-26,54439,view


In [13]:
#!!! REPORT TO FEED !!!

In [7]:
DAU = df_feed.groupby('date', as_index = False).user_id.nunique().rename(columns={'user_id':'DAU'})
Likes = df_feed.loc[df_feed.action == 'like'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'likes'})
Views = df_feed.loc[df_feed.action == 'view'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'views'})

In [8]:
report_metrics = DAU.merge(Likes).merge(Views)

In [9]:
report_metrics['CTR_%'] = round(report_metrics.likes*100/report_metrics.views,2)
#разворачиваем, чтобы 1 была вчерашняя дата
invers_report_metrics = report_metrics.sort_values('date', ascending=False)
#сохраняем данные за вчерашнюю дату
yesterday_report_metrics = invers_report_metrics.iloc[0, :]
#находим процентное отношение остальных дат к вчерашней
invers_report_metrics.iloc[:, 1:] = (invers_report_metrics.iloc[:, 1:] - invers_report_metrics.iloc[0, 1:].values.squeeze())\
                                        .div(invers_report_metrics.iloc[:, 1:])
#восстанавливаем значения за вчера
for i in range(len(yesterday_report_metrics)):
    invers_report_metrics.iloc[0, i] =yesterday_report_metrics[i]
#переводим в проценты и строки
invers_report_metrics.iloc[1:, 1:] = invers_report_metrics.iloc[1:, 1:].multiply(100).astype(float).round(2).astype(str) + '%'
invers_report_metrics['date']=invers_report_metrics['date'].astype(str)

In [10]:
value_metrics = invers_report_metrics.drop([1,2,3,4],0).values
name_metrics = report_metrics.columns.values
row_Labels = ['yesterday','1 day ago','1 week ago']

/tmp/ipykernel_107/1223561951.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  value_metrics = invers_report_metrics.drop([1,2,3,4],0).values


In [11]:
# send message to telegram
message = 'Отчет за {v0} по значениям продуктовых метрик: \nDAU, likes, views, CTR. \nТакже добавлены данные на день и неделю раньше, в процентном отношении от вчерашней даты.'\
        .format(v0 = value_metrics[0][0])
message = message + '\nСсылка на основной <a href=\"https://superset.lab.karpov.courses/superset/dashboard/327/\">дашборд!</a>'
bot.sendMessage(chat_id=chat_id, text=message, parse_mode = 'HTML')

In [12]:
sns.set(rc={'figure.figsize':(10,8), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })
#создаем таблицу
fig, ax = plt.subplots() 

plt.title('Таблица значений продуктовых метрик за вчера.')
table = ax.table( 
    cellText = value_metrics,  
    rowLabels = row_Labels,  
    colLabels = name_metrics, 
    rowColours =["steelblue"] * 10,  
    colColours =["steelblue"] * 10, 
    cellLoc ='center',  
    loc ='upper left') 
#настройка размера ячеек
ax.axis("off") 
table.auto_set_font_size(False)
table.set_fontsize(15) 
table.auto_set_column_width(col=list(range(len(report_metrics.columns))))

for row in range(1, len(row_Labels)+1):
    cell = table[row, -1]
    cell.set_height(0.1)
for col in range(0, len(name_metrics)):
    for row in range(0, len(row_Labels)+1):
        cell = table[row, col]
        cell.set_height(0.1)
plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.seek(0)
plot_object.name = 'Table_metrics.png'
plt.close()
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [13]:
sns.set(rc={'figure.figsize':(19, 13), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 'ytick.labelsize':12,'axes.titlepad': 30 })
# создание и посылка группы графиков
fig, axs = plt.subplots(2, 2)
number=0
for i in range(0,len(axs)):
    for j in range(0,len(axs)):
        number+=1
        axs[i, j].plot('date', name_metrics[number], data = report_metrics)
        axs[i, j].set_title('{v}'.format(v=name_metrics[number]))
        
plot_object = io.BytesIO()
plt.savefig(plot_object)
plot_object.seek(0)
plot_object.name = 'graph_metrics.png'
plt.close()
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [14]:
#посылка цсв файла ДФ
file_object = io.StringIO()
report_metrics.to_csv(file_object)
file_object.name = 'report_metrics.csv'
file_object.seek(0)
bot.sendDocument(chat_id=chat_id, document=file_object)

In [15]:
q = 'select toDate(time) as date, user_id, reciever_id  from simulator_20220220.message_actions where toDate(time) > today()-8 and toDate(time) < today()'

df_mess = pandahouse.read_clickhouse(q, connection=connection)

In [16]:
df_mess

,date,user_id,reciever_id
0,2022-03-23,13116,17622
1,2022-03-23,19598,120551
2,2022-03-23,114604,17622
3,2022-03-23,123681,110894
4,2022-03-23,123681,118611
...,...,...,...
109633,2022-03-26,116766,123242
109634,2022-03-26,117388,15488
109635,2022-03-26,124057,113214
109636,2022-03-26,5591,7575


In [31]:
unique_users_feed = df_feed.user_id.unique()

array([ 63830, 126860, 128603, ..., 108218,  75824,  74591], dtype=uint32)

In [19]:
df_test = df_feed.merge(df_mess, how = 'left', on = ['date','user_id'])

In [26]:
df_test.groupby('date', as_index=False).user_id.nunique()

,date,user_id
0,2022-03-20,19838
1,2022-03-21,19540
2,2022-03-22,19702
3,2022-03-23,19717
4,2022-03-24,19741
5,2022-03-25,19583
6,2022-03-26,18882


In [27]:
DAU

,date,DAU
0,2022-03-20,19838
1,2022-03-21,19540
2,2022-03-22,19702
3,2022-03-23,19717
4,2022-03-24,19741
5,2022-03-25,19583
6,2022-03-26,18882
